# 회귀분석의 결과 보고 (sklearn)

## #01. 준비작업

### [1] 패키지 참조

In [2]:
import warnings
warnings.filterwarnings(action='ignore')

import numpy as np
from pandas import read_excel, DataFrame

from sklearn.linear_model import LinearRegression       # 선형회귀분석 모듈
from sklearn.feature_selection import f_regression

# 성능 평가 지표 모듈
from sklearn.metrics import r2_score
from scipy.stats import t
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.stattools import durbin_watson

### [2] 데이터 가져오기

자동차의 속도(speed)에 따른 제동거리(dist) 조사 데이터

In [3]:
origin = read_excel("https://data.hossam.kr/mldata/cars.xlsx")
print(origin.info())
origin.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   speed   50 non-null     int64
 1   dist    50 non-null     int64
dtypes: int64(2)
memory usage: 932.0 bytes
None


,speed,dist
0,4,2
1,4,10
2,7,4
3,7,22
4,8,16


## #02. 데이터 전처리

### [1] 독립변수(들)의 이름과 종속변수의 이름을 변수에 저장

In [4]:
xnames = ['speed']
yname = 'dist'

### [2] 독립변수와 종속변수 분리

이전 예제와 동일한 결과를 만들기 위해 훈련/검증 데이터의 분할은 수행하지 않음

In [5]:
x = origin[xnames]   # 독립변수는 데이터 프레임 형태로 분리
y = origin[yname]    # 종속변수는 시리즈 형태로 분리

## #03. 회귀분석

훈련/검증 데이터를 나누지 않았으므로 검증데이터에 대한 설명력은 얻을 수 없다.

In [6]:
model = LinearRegression()
fit = model.fit(x, y)

print("계수(기울기): ", fit.coef_)
print("절편: ", fit.intercept_)
print("훈련 데이터 설명력: ", fit.score(x, y))

expr = "y = {0:0.2f} * X + {1:0.2f}".format(fit.coef_[0], fit.intercept_)
print(expr)

계수(기울기):  [3.93240876]
절편:  -17.579094890510973
훈련 데이터 설명력:  0.6510793807582509
y = 3.93 * X + -17.58


## #04. 결과보고에 필요한 값 구하기

### [1] 통계량과 p-value

#### (1) 절편과 계수를 하나의 배열로 결합

In [7]:
params = np.append(fit.intercept_, fit.coef_)
params

array([-17.57909489,   3.93240876])

#### (2) 독립변수에 상수항 추가하기

In [8]:
designX = x.copy()

# 0번째 필드에 '상수'라는 이름의 컬럼을 추가하고 값은 1로 설정함
designX.insert(0, '상수', 1)

designX.head()

,상수,speed
0,1,4
1,1,4
2,1,7
3,1,7
4,1,8


#### (3) 행렬곱 구하기

In [9]:
dot = np.dot(designX.T,designX)
dot

array([[   50,   770],
       [  770, 13228]], dtype=int64)

#### (4) 행렬곱의 역행렬

In [10]:
inv = np.linalg.inv(dot)
inv

array([[ 0.19310949, -0.01124088],
       [-0.01124088,  0.00072993]])

#### (5) 역행렬의 대각선 반환

In [11]:
dia = inv.diagonal()
dia

array([0.19310949, 0.00072993])

#### (6) 평균 제곱오차 구하기

sklearn의 API는 상수항이 고려되지 않은 MSE 구한다.

상수항이 적용된 경우이므로 API를 통한 값이 아닌 직접 구한 값이 필요하다

In [30]:
predictions = fit.predict(x)
MSE = (sum((y-predictions)**2)) / (len(designX)-len(designX.iloc[0]))
MSE

236.5316885644769

#### (7) 표준오차

In [14]:
se_b = np.sqrt(MSE * dia)
se_b

array([6.75844017, 0.41551278])

#### (8) t-value 구하기

In [15]:
ts_b = params / se_b
ts_b

array([-2.601058  ,  9.46398999])

#### (9) p-value 구하기

In [16]:
p_values = [2*(1-t.cdf(np.abs(i),(len(designX)-len(designX.iloc[0])))) for i in ts_b]
p_values

[0.01231881615380881, 1.48991929904696e-12]

#### (10) VIF 구하기

In [17]:
vif = []

for i, v in enumerate(xnames):
    j = list(origin.columns).index(v)
    vif.append(variance_inflation_factor(origin, j))

vif

[9.642207177996063]

#### (11) 결과표 구성하기

In [18]:
result_df = DataFrame({
    "종속변수": [yname] * len(xnames),
    "독립변수": xnames,
    "B": np.round(fit.coef_, 4),
    "표준오차": np.round(se_b[1:], 3),
    "β": 0,
    "t": np.round(ts_b[1:], 4),
    "유의확률": "%0.3f*" % np.round(p_values[1:], 3),
    "VIF": vif,
})

result_df

,종속변수,독립변수,B,표준오차,β,t,유의확률,VIF
0,dist,speed,3.9324,0.416,0,9.464,0.000*,9.642207


#### (12) 분석 결과 문자열

##### 잔차 구하기

$관측치 - 추정치$

In [19]:
y_pred = fit.predict(x)

# 잔차 계산
resid = y - y_pred
resid[:5]

0     3.849460
1    11.849460
2    -5.947766
3    12.052234
4     2.119825
Name: dist, dtype: float64

##### durbin_watson 구하기

In [20]:
dw = durbin_watson(resid)
dw

1.6762253234350974

##### 설명력($R^2$)

In [21]:
r2 = r2_score(y, y_pred)
r2

0.6510793807582509

##### 보정된 $R^2$

In [22]:
# 표본수
rowcount = len(origin)
rowcount

50

In [23]:
# 독립변수의 수
featurecount = len(origin.columns)
featurecount

2

In [32]:
adj_r2 = 1 - (1 - r2) * (rowcount-1) / (rowcount-featurecount)
adj_r2

0.6438102011907145

##### 통계량, p-value

In [25]:
f, p = f_regression(x, y)
f[0], p[0]

(89.5671065364682, 1.4898364962949654e-12)

##### 결과보고 문자열

In [26]:
tpl = "𝑅^2(%.3f), Adj.𝑅^2(%.3f), 𝐹(%.3f), 유의확률(%.3f), Durbin-Watson(%.3f)"
tpl % (r2, adj_r2, f[0], p[0], dw)

'𝑅^2(0.651), Adj.𝑅^2(0.644), 𝐹(89.567), 유의확률(0.000), Durbin-Watson(1.676)'

In [27]:
tpl = "%s에 대하여 %s로 예측하는 회귀분석을 실시한 결과, 이 회귀모형은 통계적으로 %s(F(%s,%s) = %0.3f, p %s 0.05)."

tpl % (
    yname,
    ",".join(xnames),
    "유의하다" if p[0] <= 0.05 else "유의하지 않다",
    len(x.columns),
    len(x.index)-2,
    f[0],
    "<=" if p[0] <= 0.05 else ">")

'dist에 대하여 speed로 예측하는 회귀분석을 실시한 결과, 이 회귀모형은 통계적으로 유의하다(F(1,48) = 89.567, p <= 0.05).'

## 참고자료(Reference)

1) https://m.blog.naver.com/pmw9440/222482746927

2) https://stackoverflow.com/questions/27928275/find-p-value-significance-in-scikit-learn-linearregression

3) https://stats.stackexchange.com/questions/85943/how-to-derive-the-standard-error-of-linear-regression-coefficient

4) https://calcworkshop.com/linear-regression/t-test/

5) https://www.google.com/url?sa=i&url=http%3A%2F%2Fwww.few.vu.nl%2F~wvanwie%2FCourses%2FHighdimensionalDataAnalysis%2FWNvanWieringen_HDDA_Lecture234_RidgeRegression_20182019.pdf&psig=AOvVaw31slQKGfkCNf2PAa3VEIGC&ust=1630070470635000&source=images&cd=vfe&ved=0CAsQjRxqFwoTCIjJworkzvICFQAAAAAdAAAAABAR